# The Price Is Right: Predicting Prices with Product Images

### Project Report

------------

**Steven Chen, Edward Chou, Richard Yang **

(Edward Chou and Richard Yang are not part of 230, but are part of 229.)



### <span style="color:red">Note: Our project is split up into several different notebooks, for ease of development. This main report will make reference to the other notebooks, where implementations and outputs are present.<span style="color:red">

The presentation is the .pptx file within the folder.

## Abstract

------------

In this work, we create deep convolutional neural networks to predict the price of a product given its image, and visualize the features that result in higher or lower price predictions. We collect two novel datasets of product images and their MSRP prices: one for bicycles, and one for cars. For our baseline models, we train linear regression on histogram of oriented gradients (HOG) and convolutional neural network (CNN) features, using PCA to reduce dimensionality. Our baseline for classification is a multi-class support vector machine (SVM) predicting price segments. For our main deep learning application, we train different CNNs for price prediction using transfer learning, with different networks and weights for regression and classification tasks. We achieve strong results, and are able to accurately predict the prices of bikes and cars. Finally, we use several state-of-the-art methods to visualize and provide insight on the image features that result in higher or lower prices.

## Introduction

------------

Online shopping is quickly becoming the norm, but the experience differs greatly from retail shopping, in which people have the opportunity to closely examine a product, weighing in the feel of a material or the scent of a cream before making a purchase decision. Online shoppers must rely entirely on the few product images to make a decision.

In this work, we build, optimize, and evaluate deep convolutional neural networks using transfer learning, that can predict prices based on product images This model can be used by both buyers and sellers to suggest fair prices for products, or warn of inaccurate or unreasonable pricing. In this work, we also visualize which features tend to result in predicted higher or lower prices, using sliding windows, saliency maps, and class activation maps. Our proposed model can help sellers increase the perceived value of their products on shopping websites, helping guide product design, photo selection, and product description to improve a buyer’s impression, as well as provide valuations of products for bidding websites using just a single image.

## Related Work

---------------

Computer vision and supervised machine learning have been used in conjunction for a variety of pricing and regression tasks. Various works have used computer vision methods to perform regression on image data. Early work has used supervised learning to predict attractiveness given labeled faces [3]. This early work demonstrated the efficacy of machine regression on human-centric evaluation tasks. Recent works have predicted age using face images [4, 5], and housing prices with satellite imagery [7, 15], tasks which are traditionally difficult for humans. In contrast to these works, we focus on the task of prediction using images of consumer products, with novel datasets curated specifically for this purpose.

ClickToPrice [8] proposes the most similar concept to our work. In ClickToPrice, Maurya explores the predictive power of product images for prices. Our project is similar in that we use machine learning to predict the price of a product. However, [8] uses very basic techniques to classify the general product category (e.g., towels, soap, shoes), and uses that categorization alone to predict the average price for each product. We argue that such a model is functionally equivalent to object recognition, and is not suited for price prediction. Our models are specifically designed for fine-grained price prediction and are more sophisticated and novel in technical implementation. In particular, we tackle the difficult problem of price prediction for items of the same type using models and methods specifically designed for price regression and classification.

Recent research has delved into methods for visualizing what features and images CNNs use to make their predictions. Zeiler and Fergus [16] learn what visual features maximize activations in a convolutional network, and use obscuring sliding windows to determine which features influence prediction score. Yosinski et al. [14] build live visualization of hidden unit activations, allowing for easier discovery of the inner workings of CNNs. Zhou et al. [17] use global average pooling to visualize what regions of images are most responsible for classification predictions that a neural network makes. In contrast, Simonyan et al. [12] generate images that maximize the class score predicted by an object recognition network. We experiment with a subset of these methods to visualize the features that result in higher or lower prices for products.

## Datasets

------------

In this work, we choose to use cars and bikes as target product datasets, due to the wide visual variances in bike and car models, close visual correlations to their prices, and relevance of online shopping for cars and bikes. We collect and clean images for both bikes and cars that are visually consistent, and match images with accurate MSRP price figures.

Our first dataset, **bikes**, is curated from Bicycle Blue Book, a database of catalog data from high-end bicycle shops. We collect images, model names, and manufacturer-suggested retail prices (MSRP) from the listings. We preprocess the images by filtering out low-resolution images, removing images with noisy backgrounds, and resizing the remaining to 224 by 224 pixels. Our final dataset contains cleaned, solid white background, side view bike images with the same orientation (see Figure below). The dataset consists of 21,843 images, each with a model name and MSRP price.

Our second dataset, **cars**, is a dataset of passenger vehicle images along with their MSRP prices. We retrieve price data from Kaggle (www.kaggle.com/jshih7/car-price-prediction), containing make, model, and year, along with prices sourced from Edmunds. We join these prices on image data collected from Google Images, using search terms consisting of the make, model, year, and the keyword "Angular Front View". We clean the car images, keeping images without backgrounds and in the same orientation. Our final dataset consists of 1,400 images, each with a corresponding model name and MSRP price.

The bike dataset prices range between \$70 and \$17,000, and the car dataset prices range between \$12,000 and \$2,000,000. See figures below for histogram distributions of prices. The prices closely follow an exponential distribution (most prices at the low or mid range, few prices at the upper range), which is consistent with the real-world economy, where there are significantly more models at the modest and regular price segments than at the luxury segment.

**Note:** Our datasets are quite large, and are not included with this notebook.

Samples from the bikes dataset:
![bike dataset](http://www.stevenzc.com/assets/cs230/bike_montage.jpg)


Samples from the cars dataset:
![bike dataset](http://www.stevenzc.com/assets/cs230/car_montage.jpg)

Histogram of the bikes dataset prices:
![bike dataset](http://www.stevenzc.com/assets/cs230/bike_histogram.png)


Histogram of the the cars dataset prices:
![bike dataset](http://www.stevenzc.com/assets/cs230/car_histogram.png)

## Baseline Approaches

We predict prices using both regression and classification. Our regression model is our main focus for accuracy, because it directly predicts the price. We use the classification model to help create certain visualizations.

------------

### Linear Regression

We use linear regression as the regression baseline. We train linear regression using histogram of oriented gradients (HOG) [2], as well as using CNN features generated using the final convolutional layer of VGG-16 [13]. In both cases, we reduce the dimensionality using PCA to 200 and 256 dimensions respectively before training.

**<span style="color:red">The notebooks Linear_Regression_CNN_Features, Linear_Regression_HOG_Features, Car_Linear_Regression_CNN_Features, and Car_Linear_Regression_HOG_Features contain these baselines.<span style="color:red">**

### Multiclass SVM

Our baseline for classification is a one vs. one support vector machine, using price segments as categories. The multiclass SVM is trained on the PCA-reduced CNN features that were used to train the second linear regression baseline.

**<span style="color:red">The notebooks SVM_bikes and SVM_cars contain these baselines.<span style="color:red">**

## Deep Learning Model

------------------------

Our main deep learning models are two transfer-learned deep convolutional neural networks, sharing the same general architecture but trained for different tasks (regression and classification).

Training an entire neural network would likely be impractical due to the smaller size of our datasets (especially cars). Instead, taking advice from Professor Ng's lectures, we use transfer learning, where we take advantage of the weights learned by an existing CNN, removing certain layers and adding our own to train to suit our specific tasks.

In particular, we use the pretrained ImageNet object recognition network VGG-16 [13], a very deep convolutional neural network consisting of many layers of small convolution and pooling filters, followed by fully connected layers and a softmax output. 

Throughout our project, we use the Keras [1] framework, which we were introduced to in CS230. Keras is great for building complex CNNs due to its succinct and simple syntax and availability of many great implementations of optimizers, vizualizers, and other tools.

For our network, we load the architecture and weights of VGG-16, and remove the network's softmax output along with its fully connected layers. We set the convolution and pooling layers to be fixed (non-trainable), and add our own fully connected layer of 256 units.

We use two different output layers, one for each task. For regression, we use a linear output. For classification, we use a softmax. 

## Implementation and Tuning

We implement our deep learning models using Keras, training them inside of FloydHub through the Jupyter notebook interface. 

---------------------

### CNN for Price Regression

**<span style="color:red">The notebooks Bikes_CNN and Cars_CNN contain the regression CNN implementations described in this section.<span style="color:red">**

While training our transfer learning model, we used several techniques to tune our weights and parameters. For regression on the both datasets, we obtained the highest performance with the RMSprop optimizer on the mean squared error, which divides learning rate by an exponentially decaying average of squared gradients. We also use dropout, dropping the effect of random hidden units during training, to help reduce overfitting. For parameter selection, we first tune parameters over a log scale and find the best candidates, and then fine tune over a smaller range around the candidates. We tuned the learning rate, minibatch size, number of hidden units, and number of training epochs.

Our final parameter values for cars were: 0.2 dropout on the dense layer, 0.01 learning rate, 256 hidden units, minibatch size of 64, and 30 epochs of training. We used FloydHub, which after some setup time proved to be extremely helpful in quickly training our model. FloydHub would sometimes preempt our job, so we would checkpoint the model at certain stages of training. For bikes, the hyperparameter values were the same except for learning rate, which was 0.01, and number of epochs, which was 20. The networks took about an hour to train.

### CNN for Price Classification

**<span style="color:red">The notebooks Bikes_CNN_Multiclass and Cars_CNN_Multiclass contain the classification CNN implementations described in this section.<span style="color:red">**

Our classification CNNs were trained in a similar format to the regression CNNs. For car classification, we used the Adam optimizer with categorical crossentropy loss and a learning rate of 0.0003. We used a minibatch size of 70 and trained for 25 epochs. For bikes, we used the RMSprop optimizer with the same loss, using a learning rate of 0.0001. We use a minibatch size of 64 and train for 30 epochs.

## Evaluation Results

--------------------

We split our datasets into training and testing splits, which are consistent across all models to ensure a fair comparison. Models are trained and optimized on the training set and then evaluated on the test set. To create the split for both datasets, we first shuffle the data points, then assign 90 percent of the points to the training set, and the remaining 10 percent to the test set. We use a large percentage for training because our datasets are large enough (over 1,000 for cars and over 20,000 for bikes) to have a representative test split as a small percentage, and so that our deep learning models have more data to learn on.

### Regression

We use three different metrics to evaluate and compare the performance of our models on price regression: root mean squared error (RMSE), mean absolute error (MAE), and coefficient of determination ($R^2$). 

RMSE measures the root of average squared error between the predicted and actual price, while MAE is interpreted as the average absolute difference in price. Lower values are better for both. Coefficient of determination measures goodness of fit through the proportion of variance explained by the model, and lies between 0 and 1, where higher values are better.

We report results of our two linear regression models and the deep CNN in the tables below, alongside a naive baseline that always predicts the average price. All models significantly outperform the naive baseline, with linear regression on CNN features showing a margin of improvement over histogram of oriented gradients, likely due to the CNN features providing more discriminative visual cues. The deep CNNs in particular have very strong performance, significantly outperforming the other models in every metric. The deep CNN models reduce MAE by almost a factor of two for bikes and a more than a factor of 3 for bikes.

| Bikes Regression        | RMSE           | MAE  |  R2 |
| :------------- |-------------: | -----:| -----:|
| Average Baseline      | 1810.19 | 1318.53 | 0.00 |
| LinReg (HOG Features)      | 1274.99      |   833.02 | 0.50 |
| LinReg (CNN Features) | 1054.67      | 712.63 | 0.66 |
| **Deep CNN** | **747.42** | **405.50** | **0.83** |

| Cars Regression        | RMSE           | MAE  |  R2 |
| :------------- |-------------: | -----:| -----:|
| Average Baseline      | 76240.41 | 44410.57 | 0.00 |
| LinReg (HOG Features)      | 41898.48      |  27588.70 | 0.70 |
| LinReg (CNN Features) | 37808.84      | 23929.67 | 0.75 |
| **Deep CNN** | **12363.65** | **7477.74** | **0.97** |

### Classification

For classification, we assign a class segments to each example using price cutoffs roughly corresponding to percentiles of price. We assign labels of 25, 50, 75, 100 for the bikes dataset (4 classes), and 20, 40, 60, 80, 100 for the cars dataset (5 classes). 

We evaluate our classification models based on three metrics: precision, recall, and the F-1 score. Precision measures the percent of positive predictions that were correct. Recall measures the percent of positive predictions out all positive examples. The F-1 score is calculated as the harmonic mean of precision and recall. Our deep learning model outperforms the SVM on bikes significantly, and achieves similar performance on the cars dataset.

| Model        | Precision           | Recall  |  F1 |
| :------------- |-------------: | -----:| -----:|
| SVM on bikes      | 0.80 | 0.45 | 0.43 |
| Deep CNN on bikes      | 0.74      |  0.75 | 0.74 |
| SVM on cars | 0.83      | 0.82 | 0.82 |
| Deep CNN on cars | 0.82 | 0.82 | 0.82 |

## Visualizations

-----------------

### Sliding Window Heatmaps

**<span style="color:red">The notebooks Bike_Sliding_Window and Car_Sliding_Window contain the implementations.<span style="color:red">**

We use obscuring sliding windows on the deep CNN with linear output (regression) to determine which features of input images are important to determining the predicted price, in the vein of [16]. We slide a 28 by 28 pixel window over an input image, obscuring that area of the image by replacing the pixel values with the average value for the network. We then run the obscured images through the network, and compare the predicted price of the obscured images to the original predicted price.

We visualize these changes using a heatmap, where each square of the heatmap corresponds to the region of the image that was obscured. Our best example is shown below: when the training wheel on the bike is obscured, the predicted price of the bike increases by 150 dollars!

![bike saliency maps](http://www.stevenzc.com/assets/cs230/kidbike.jpg)

![car saliency maps](http://www.stevenzc.com/assets/cs230/bike_heatmap.png)

## Saliency Maps

**<span style="color:red">The notebook titled visualizations contain the implementations.<span style="color:red">**

For our classification network, we create saliency maps to visualize how individual pixels contribute to the output. In [12], the authors show that the class score $S_C$ of an image $I$ can be approximated by the first-order Taylor expansion $S_C(I) = w^TI+b$, where $w$ is calculated by taking the gradient of the output class with respect to the input image. $w$ corresponds to the weight of each pixel in $I$, and shows much each pixel contributes to the output classification.

We use the Keras-vis toolkit [6] to help us generate the saliency maps. Features such as the seat and handlebars of the bikes, as well as the brand and body contours of the cars, are particularly significant.

![bike saliency maps](http://www.stevenzc.com/assets/cs230/bike_saliency_map_montage.png)

![car saliency maps](http://www.stevenzc.com/assets/cs230/car_saliency_map_montage.png)

## Class Activation Maps

**<span style="color:red">The notebook titled visualizations contain the implementations.<span style="color:red">**

For our classification network, we create saliency maps to visualize how individual pixels contribute to the output. In [12], the authors show that the class score $S_C$ of an image $I$ can be approximated by the first-order Taylor expansion $S_C(I) = w^TI+b$, where $w$ is calculated by taking the gradient of the output class with respect to the input image. $w$ corresponds to the weight of each pixel in $I$, and shows much each pixel contributes to the output classification.

We use the Keras-vis toolkit [6] to help us generate the class activation maps. For bikes, the class activation maps highlight that the significant areas are the brakes, gears, seats, and wheels. For cars, the method highlights the doors (seeing if the car is a 2 door or 4 door), as well as the open top for convertibles.

![bike cam montage](http://www.stevenzc.com/assets/cs230/bikes_cam_montage.png)

![car cam montage](http://www.stevenzc.com/assets/cs230/cars_cam_montage.png)

## Future Work

---------------------

By analyzing the results of our models on the cars and bikes datasets, we have identified potential real-world applications with the techniques we have developed. Using feature visualization, sellers can determine what features of objects are correlated to higher or lower prices, and use this knowledge to help guide visual design. For instance, some cars are specifically targeted towards luxury consumers, and having the design convey high value is important in a purchasing decision. Similar techniques could be used for other luxury goods, like watches, food, and yachts.

With the right datasets, our model can be extended to assist valuations and auction pricing. For example, by training on datasets that include used car resale values, a pricing model can help customers evaluate the visual value of a car in order to help guide their decisions. This model can also be applied to auction sites such as eBay for recommendations of starting bids, and provide a tool to online resellers for choosing better photo quality.

## References

-----------

![references](http://www.stevenzc.com/assets/cs230/refs.jpeg)